In [1]:
# data prepare
import glob,re,os,sys
os.chdir('C:/kongjy_special/实习工作内容总结/201805-201808华泰证券/Task4/')
sys.path.append('C:\\kongjy_special\\实习工作内容总结\\201805-201808华泰证券\\Task4')
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import time
import TranFeature,Tools_kjy
import warnings

warnings.filterwarnings('ignore')
pd.options.display.width = 200

# 文件名
TickNameList = glob.glob('Java_log_orders_600519/Tick*.csv')
TranNameList = glob.glob('Java_log_orders_600519/Tran*.csv')
# TestName是 开平仓点 的csv，需要和Tick以及Tran保持时间对齐
DateList = [re.search('\d{8}',i)[0] for i in TickNameList if re.search('\d{8}',i)]  
TestNameFun = lambda x: "Java_log_orders_600519\\600519.SH_back_test_info_" + x +".csv"
TestNameList = [TestNameFun(i) for i in DateList]
print (TickNameList[0])
print (TranNameList[0])
print (TestNameList[0])


Java_log_orders_600519\Tick20180502.csv
Java_log_orders_600519\Tran20180502.csv
Java_log_orders_600519\600519.SH_back_test_info_20180502.csv


In [2]:

# 收集所有数据
DataSave = pd.DataFrame()
# 数据读入 和 预处理
for date_index in range(len(TickNameList)):
    date = re.search('\d{8}',TickNameList[date_index])[0]
    tempTick = pd.read_csv(TickNameList[date_index])
    tempTran = pd.read_csv(TranNameList[date_index])
    tempTest = pd.read_csv(TestNameList[date_index])
    # Tick数据处理
    tempTick = tempTick.rename(columns={"Unnamed: 0":"Timestamp"})
    tempTick['Timestamp'] = pd.to_datetime(tempTick['Timestamp'])
    tempTick['median_5'] = (tempTick['Price'].rolling(window=5).median()).shift(-2).fillna(method='pad')
    tempTick['median_5'],_ = Tools_kjy.smooth(tempTick['median_5'])
    # 开平仓点提取
    tickindex = tempTest[tempTest.order != " "].index
    print ("Date : %s with %s data"%(date,len(tickindex)))
    # # Tran 数据处理
    tempTran = tempTran.rename(columns={"Unnamed: 0":"Timestamp"})
    tempTran['Timestamp'] = pd.to_datetime(tempTran['Timestamp'])
    FeatureTran = TranFeature.TranFeature(tempTran,tempTick)
    tempTick = FeatureTran._Tran_features()
    
    # past change
    def past_change(x):
        tempTick['Past_diff'+str(x)] = tempTick['Price'].diff(x).fillna(0)
    past_change(1);past_change(2);past_change(3);past_change(4);past_change(5);past_change(6)

    # future change
    tempTick['diff_3Tick'] = tempTick['Price'].diff(3).shift(-3).fillna(0)
    tempTick['diff_1Tick'] = tempTick['Price'].diff(1).shift(-1).fillna(0)
    tempTick['isadd_3Tick'] = tempTick['diff_3Tick'].apply(Tools_kjy.trinary)
    tempTick['isadd_1Tick'] = tempTick['diff_1Tick'].apply(Tools_kjy.trinary)
    # 开平仓点保存
    DataSave = DataSave.append(tempTick.loc[tickindex,:])
 
print ('done!')
DataSave = DataSave.reset_index(drop=True)

Date : 20180502 with 119 data
_drop_Contunous costs 0.115 seconds!
Date : 20180503 with 103 data
_drop_Contunous costs 0.070 seconds!
Date : 20180504 with 107 data
_drop_Contunous costs 0.085 seconds!
Date : 20180507 with 122 data
_drop_Contunous costs 0.128 seconds!
Date : 20180508 with 91 data
_drop_Contunous costs 0.110 seconds!
Date : 20180509 with 75 data
_drop_Contunous costs 0.077 seconds!
Date : 20180510 with 54 data
_drop_Contunous costs 0.068 seconds!
Date : 20180511 with 135 data
_drop_Contunous costs 0.070 seconds!
Date : 20180514 with 123 data
_drop_Contunous costs 0.090 seconds!
Date : 20180515 with 144 data
_drop_Contunous costs 0.072 seconds!
Date : 20180516 with 86 data
_drop_Contunous costs 0.061 seconds!
Date : 20180517 with 108 data
_drop_Contunous costs 0.065 seconds!
Date : 20180518 with 96 data
_drop_Contunous costs 0.064 seconds!
Date : 20180521 with 62 data
_drop_Contunous costs 0.067 seconds!
done!


In [20]:
DataSave.columns

Index(['Timestamp', 'AskP0', 'AskP1', 'AskP2', 'AskP3', 'AskP4', 'AskP5', 'AskP6', 'AskP7', 'AskP8', 'AskP9', 'AskV0', 'AskV1', 'AskV2', 'AskV3', 'AskV4', 'AskV5', 'AskV6', 'AskV7', 'AskV8', 'AskV9',
       'BidP0', 'BidP1', 'BidP2', 'BidP3', 'BidP4', 'BidP5', 'BidP6', 'BidP7', 'BidP8', 'BidP9', 'BidV0', 'BidV1', 'BidV2', 'BidV3', 'BidV4', 'BidV5', 'BidV6', 'BidV7', 'BidV8', 'BidV9', 'Date',
       'High', 'Low', 'Price', 'Time', 'Turnover', 'Volume', 'MatchItem', 'median_5', 'LowPrice', 'HighPrice', 'MedianPrice', 'CountPrice', 'MedianBid', 'MedianAsk', 'CountBid', 'CountAsk',
       'Past_diff1', 'Past_diff2', 'Past_diff3', 'Past_diff4', 'Past_diff5', 'Past_diff6', 'diff_3Tick', 'diff_1Tick', 'isadd_3Tick', 'isadd_1Tick'],
      dtype='object')

In [4]:
DataSave['diff_1Tick'] = DataSave['diff_1Tick'].clip(-0.3,0.3)
DataSave['diff_3Tick'].value_counts(bins=10,sort=False)

(-0.302, -0.24]    310
(-0.24, -0.18]      58
(-0.18, -0.12]      80
(-0.12, -0.06]      89
(-0.06, 0.0]       235
(0.0, 0.06]        119
(0.06, 0.12]        87
(0.12, 0.18]        73
(0.18, 0.24]        77
(0.24, 0.3]        297
Name: diff_3Tick, dtype: int64

In [5]:
# 选取其中两类来研究
# check1 : (-0.302, -0.24]    310
# check2 : (0.24, 0.3]        297
# check3 : (-0.06, 0.0]       235  + (0.0, 0.06]        119

check1 = DataSave[DataSave['diff_3Tick'] < -0.24].index
check2 = DataSave[DataSave['diff_3Tick'] > 0.24].index
check3 = DataSave[DataSave['diff_3Tick'] < 0.06 ].index & DataSave[DataSave['diff_3Tick'] > -0.06].index
DataSave['CountBid'] = DataSave['CountBid'].clip(0,12).apply(int)
DataSave['CountAsk'] = DataSave['CountAsk'].clip(0,12).apply(int)



In [8]:
# check1 : (-0.302, -0.24]    310
print ('降低最大的部分，占比%.3f:'%(len(check1)/DataSave.shape[0]))
print (DataSave.loc[check1,['CountBid','CountAsk','diff_3Tick','diff_1Tick','Turnover',
                                'Past_diff2','Past_diff3','Past_diff4','Past_diff6']].head(20),'\n')

print ('统计这里面CountBid 的分布')
print (DataSave.loc[check1,'CountBid'].value_counts(),'\n')
print ('统计这里面CountAsk 的分布')
print (DataSave.loc[check1,'CountAsk'].value_counts(),'\n')

print ('从其中拿出CountBid 和 CountAsk为固定值的部分')
check1_BA35 = check1 & DataSave[DataSave['CountBid'] == 1].index & DataSave[DataSave['CountAsk'] == 1].index 
print (DataSave.loc[check1_BA35,['CountBid','CountAsk','diff_3Tick','diff_1Tick','Turnover',
                                'Past_diff2','Past_diff3','Past_diff4','Past_diff6']].head(20),'\n')

降低最大的部分，占比0.218:
     CountBid  CountAsk  diff_3Tick  diff_1Tick   Turnover  Past_diff2  Past_diff3  Past_diff4  Past_diff6
0          12        12       -0.30        0.30  4995578.0       -1.00       -1.00       -1.30       -1.88
2          12         3       -0.30       -0.30  9206900.0        1.97        0.50        1.50        0.50
4           8         3       -0.30       -0.03  1617071.0       -0.50       -0.50        0.00       -1.00
16         10        12       -0.30        0.00  3618419.0        0.50        0.70        0.50        0.70
17         12         2       -0.30       -0.10  2271472.0       -0.76       -0.70       -0.77       -0.78
18          6         3       -0.30       -0.10  1038269.0       -0.10       -0.86       -0.80       -0.10
22          4         5       -0.30        0.00  1132920.0        0.00        0.49        0.49        0.10
31          9         2       -0.30       -0.03   863175.0       -0.02       -0.01       -0.32       -1.02
57          3       

In [9]:
# check2 : (0.24, 0.3]        297
print ('增长最大的部分，占比%.3f:'%(len(check2)/DataSave.shape[0]))
print (DataSave.loc[check2,['CountBid','CountAsk','diff_3Tick','diff_1Tick','Turnover',
                                'Past_diff2','Past_diff3','Past_diff4','Past_diff6']].head(20),'\n')

print ('统计这里面CountBid 的分布')
print (DataSave.loc[check2,'CountBid'].value_counts(),'\n')
print ('统计这里面CountAsk 的分布')
print (DataSave.loc[check2,'CountAsk'].value_counts(),'\n')

增长最大的部分，占比0.208:
     CountBid  CountAsk  diff_3Tick  diff_1Tick    Turnover  Past_diff2  Past_diff3  Past_diff4  Past_diff6
1          12         9        0.30        0.30   3367505.0       -1.67       -0.67       -1.67       -1.67
8           6         6        0.30        0.00   1678389.0       -0.10        0.00       -0.52       -0.43
10         12         9        0.30        0.20   4004630.0       -0.30        0.00        0.10       -0.30
11          8         8        0.30        0.26  10786985.0       -0.10       -0.10        0.20        0.30
12         12        12        0.30        0.00  12164703.0        0.46        0.16        0.16        0.56
14         12         6        0.30        0.00   5296722.0        0.00        0.00        0.20       -0.40
24          9         8        0.30       -0.30   2853931.0        0.00        0.00        0.00        0.49
34          3         6        0.30        0.30   1242679.0       -1.25        0.00        0.00        0.00
49         

In [10]:
# check3 : (-0.06, 0.0]       235  + (0.0, 0.06]        119
print ('增长最少的部分，占比%.3f:'%(len(check3)/DataSave.shape[0]))
print (DataSave.loc[check2,['CountBid','CountAsk','diff_3Tick','diff_1Tick','Turnover',
                                'Past_diff2','Past_diff3','Past_diff4','Past_diff6']].head(20),'\n')

print ('统计这里面CountBid 的分布')
print (DataSave.loc[check3,'CountBid'].value_counts(),'\n')
print ('统计这里面CountAsk 的分布')
print (DataSave.loc[check3,'CountAsk'].value_counts(),'\n')

增长最少的部分，占比0.248:
     CountBid  CountAsk  diff_3Tick  diff_1Tick    Turnover  Past_diff2  Past_diff3  Past_diff4  Past_diff6
1          12         9        0.30        0.30   3367505.0       -1.67       -0.67       -1.67       -1.67
8           6         6        0.30        0.00   1678389.0       -0.10        0.00       -0.52       -0.43
10         12         9        0.30        0.20   4004630.0       -0.30        0.00        0.10       -0.30
11          8         8        0.30        0.26  10786985.0       -0.10       -0.10        0.20        0.30
12         12        12        0.30        0.00  12164703.0        0.46        0.16        0.16        0.56
14         12         6        0.30        0.00   5296722.0        0.00        0.00        0.20       -0.40
24          9         8        0.30       -0.30   2853931.0        0.00        0.00        0.00        0.49
34          3         6        0.30        0.30   1242679.0       -1.25        0.00        0.00        0.00
49         